# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [11]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [12]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [13]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [27]:
openai = OpenAI()
llama_via_open_ai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [20]:
system_prompt = "You are a programming assistant that analyzes user questions and creates detailed technical explanations.\
Expect the questions to be related to coding or contain actual lines of code. Respond in markdown.\
"

In [21]:
def messages_for(question):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

In [22]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [25]:
# Get gpt-4o-mini to answer, with streaming
def gpt_ask_question(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages_for(question),
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
gpt_ask_question(question)

The line of code you provided uses a combination of a generator and a set comprehension in Python. Let's break it down step by step to understand its functionality and purpose:

### Components of the Code

1. **Set Comprehension**: 
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   - **`books`**: This is assumed to be an iterable (like a list or a set) containing book dictionaries. Each dictionary presumably represents a book with various attributes, one of which is the "author".
   - **`book.get("author")`**: This method retrieves the value for the key "author" from each book dictionary. If the key doesn't exist or its value is `None`, it will return `None`.
   - **`if book.get("author")`**: This condition checks to ensure that only books with a defined author are included in the resulting set. It filters out entries that do not have valid authors (i.e., `None` values or missing keys).

2. **Yielding Values with `yield from`**:
   ```python
   yield from <iterator>
   ```
   - **`yield from`**: This keyword is used in Python to delegate part of a generator's operations to another iterable (like a list, generator, or in this case, a set).
   - It effectively yields one item at a time from the iterable it is provided. This is useful for creating a generator that can iterate over multiple sources of values efficiently.

### Overall Functionality

Putting this all together:

- The code creates a set of unique authors from a collection of book dictionaries. 
- The set comprehension runs through all the `books`, checks each book for an author, and collects the authors into a set (ensuring uniqueness).
- The `yield from` keyword is then used to yield each author one by one as the generator is iterated over. 

### Example Use Case

If you have a list of books, where each book is a dictionary like this:

```python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three"},  # No 'author' key
    {"title": "Book Four", "author": "Author A"},  # Duplicate author
]
```

Using your code:

```python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}
```

If you iterate over `get_authors(books)`, it would yield:

```
'Author A'
'Author B'
```

### Conclusion

The purpose of the code is to produce an iterable of unique authors from a list of books, thus avoiding duplicates and omitting any books that do not have an specified author. The use of `yield from` makes it possible to yield items iteratively, which is efficient for large datasets or streams of data.

In [39]:
# Get Llama 3.2 to answer

def llama_ask_question(question):
    stream = llama_via_open_ai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages_for(question),
        # messages=[{"role": "user", "content": question}],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [40]:
llama_ask_question(question)

**Delegating Generator Yield**
================================

The given line of code is using a feature called "delegation" or "sub-generating", which allows us to bypass the normal yield syntax. Let's break it down:

### The Code

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Explanation

This line of code uses several features introduced in Python 3.x, specifically:

*   **Generators**: defined by the `yield` keyword
*   **Dictionaries**: can be iterated over using a dictionary comprehension
*   **Delegation through `yield from`**

Here's how it works:

1.  `{book.get("author") for book in books if book.get("author")}`: This is a dictionary comprehension.
    *   It generates a dictionary where each key is an author and the corresponding value is a list of their authored books.
    *   The `if` condition filters out any books that do not contain an "author" key.
2.  `yield from`: This keyword allows us to delegate control to another generator or coroutine, effectively combining them.
3.  Since we're using a dictionary comprehension as the delegating object, it essentially turns into a "generator expression wrapped around a list".

### How It Works Behind the Scenes

When you iterate over this delegated yield, Python executes:

1.  The original code iterates over `books`.
2.  For each book in `books`, it checks if that book contains an author by calling `book.get("author")`. If yes, it contributes their "author" to the dictionary comprehension.
3.  After generating dictionaries for all authors, Python yields from these generators.

### Why It's Used

This delegation technique helps to:

*   Combine multiple generator expressions into a concise list.
*   Simplify complex iteration logic.
*   Avoid creating large lists in memory when dealing with huge data inputs.

**Example Use Case**

```python
def get_authors(authorized_books):
    """Retrieve all authors that wrote authorized books"""
    
    # Assuming 'books' is a dictionary where values are dictionaries of book metadata
    
    def books_get_author(book_dict):
        """Use yield from to fetch and filter author info"""
        
        yield from {book.get("author", None) for book in book_dict["books"] if "author" in book}
    
    return set([book for book, authors in authorized_books.items() if authors])
    

# Usage example
books = {
    "Book 1": {"author": "John", "title": "Intro to Python"},
    "Book 2": {"author": "Jane", "title": "Python Fundamentals"}
}

authored_books = get_authors(books)

for book_title, authors in authored_books.items():
    if authors:
        print(f"Title: {book_title}, Author(s): {[e for e in authors]}")
```